# Survival analysis para la empresa JAMPP

El objetivo es analizar los datos de la empresa JAMPP para saber cuando un usuario volverá a conectarse a una app, dado sus datos de conexiones previas. La segunda predicción que intentaremos hacer es saber cuando volverá a "convertir", es decir comprar la app de la cual tiene una publicidad en su movil a un momento dado.



In [460]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import lifelines
import math
import category_encoders as ce
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import xgboost

In [2]:
events = pd.read_csv('./events.csv')

/Users/yorickvinesse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
clicks = pd.read_csv('./clicks.csv')

/Users/yorickvinesse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
installs = pd.read_csv('./installs.csv')

/Users/yorickvinesse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
installs.dtypes

created                object
application_id          int64
ref_type                int64
ref_hash                int64
click_hash             object
attributed               bool
implicit                 bool
device_countrycode      int64
device_brand          float64
device_model          float64
session_user_agent     object
user_agent             object
event_uuid             object
kind                   object
wifi                   object
trans_id               object
ip_address              int64
device_language       float64
dtype: object

In [6]:
events.dtypes

index                   int64
date                   object
event_id                int64
ref_type                int64
ref_hash                int64
application_id          int64
attributed               bool
device_countrycode      int64
device_os_version     float64
device_brand          float64
device_model          float64
device_city           float64
session_user_agent    float64
trans_id               object
user_agent            float64
event_uuid             object
carrier               float64
kind                  float64
device_os             float64
wifi                     bool
connection_type        object
ip_address              int64
device_language       float64
dtype: object

# Que datos nos conviene usar?

Acá estamos haciendo un _survival analysis_ entonces se necesita mostrar el momento al cual ocurrió un evento. En nuestro caso, si un evento aparece en el dataset de _auctions_, es porque ocurrió. El survival analysis se enfoca en la primera vez que ocurre un evento, puesto que hace una regresión de un set de caractéristicas en frente a una fecha de ocurrencia del evento.

Pero es cierto que muchos usuarios, aca representados por sus *device_id* aparecen varias veces en la lista de subastas (_auctions_), considerando que 475338 *device_id* únicos se pueden contar adentro del set de _auctions_ que almacena 47409528 observaciones distintas (100 veces más).

Dado eso, parecería inoportuno no considerar la característica de repetición de aparición de los usuarios cuando hacemos la predicción de su reaparición más adelante.

Podemos tratar de analizar datos como la cantidad de clicks hecha por un usuario antes de llevar a cabo una instalación, o la cantidad de veces que apareció en una subasta.

In [3]:
auctions.head(5)

,date,device_id,ref_type_id,source_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0


In [23]:
auctions.date.max()

'2019-04-26 23:59:59.969518'

In [33]:
events.count()

index                 7744581
date                  7744581
event_id              7744581
ref_type              7744581
ref_hash              7744581
application_id        7744581
attributed            7744581
device_countrycode    7744581
device_os_version     2332975
device_brand          2553424
device_model          5668092
device_city           1894935
session_user_agent    7702301
trans_id                37642
user_agent            3341483
event_uuid            7714809
carrier               1925901
kind                  7714809
device_os             1870190
wifi                  7744581
connection_type       1809296
ip_address            7744581
device_language       5665409
dtype: int64

# Cuandó instalará un usuario dado?

### Desarrollo alrededor del dataset _installs_ y _events_

In [7]:
eventsCurated = events.drop(columns = ['ref_type', 'session_user_agent', 'device_os_version', 'device_brand',\
                                       'device_model', 'trans_id', 'user_agent', 'ip_address', 'device_countrycode'])

In [8]:
eventsCurated.head(2)

,index,date,event_id,ref_hash,application_id,attributed,device_city,event_uuid,carrier,kind,device_os,wifi,connection_type,device_language
0,2130678,2019-04-20 01:42:49.120,0,5857744372586891366,210,False,NaN,5b506964-5f47-4b28-a8c2-8a92d6c23379,NaN,5.882882e+18,NaN,False,NaN,3.301378e+18
1,2130680,2019-04-20 01:42:49.340,1,7642521036780133571,210,False,NaN,f1fb9d15-1a7b-4116-8d3b-c4c403e197e2,NaN,4.017674e+18,NaN,False,NaN,NaN


In [ ]:
installsSinDoble = installs[]

In [150]:
installsCurated = installs.drop(columns = ['application_id', 'ref_type', \
                                           'click_hash', 'attributed', 'device_brand', 'device_model',\
                                           'session_user_agent', 'user_agent', 'kind', 'trans_id',\
                                          'ip_address', 'device_language', 'device_countrycode'])

In [151]:
installsCurated['installed'] = 1

In [152]:
installsCurated.head(2)

,created,ref_hash,implicit,event_uuid,wifi,installed
0,2019-04-24 06:23:29.495,4716708407362582887,True,79837499-2f2a-4605-a663-e322f759424f,NaN,1
1,2019-04-24 02:06:01.032,7143568733100935872,False,NaN,NaN,1


In [153]:
installsCurated = installsCurated.sort_values(['ref_hash', 'created'], ascending = [True, True])

In [154]:
installsCurated.loc[:,'created'] = pd.to_datetime(installsCurated['created'], errors = 'coerce')

In [155]:
installsCurated.dtypes

created       datetime64[ns]
ref_hash               int64
implicit                bool
event_uuid            object
wifi                  object
installed              int64
dtype: object

In [156]:
installsCurated.count()

created       481511
ref_hash      481511
implicit      481511
event_uuid    103168
wifi          294829
installed     481511
dtype: int64

## Vamos a suprimir los installs que parecen ser los mismos, es decir lo que tienen el mismo hash y un tiempo de instalacion dentro de 1 minuto.

In [157]:
installsCurated['is_replica'] = False

In [158]:
segundo = np.timedelta64(1, 's')

In [147]:
def isNaN(num):
    return num != num

In [161]:
installsCurated.head(25)

,created,ref_hash,implicit,event_uuid,wifi,installed,is_replica
46742,2019-04-21 19:17:47.657,40621409780134,False,NaN,NaN,1,False
398921,2019-04-18 21:11:50.326,41863526108385,True,2f8be0cc-297e-4c9c-a097-1096aa5824b5,False,1,False
249204,2019-04-18 21:11:51.966,41863526108385,False,NaN,False,1,False
112218,2019-04-18 21:17:11.946,41863526108385,False,NaN,False,1,False
112200,2019-04-18 21:17:16.531,41863526108385,True,f07fd6db-2f55-4539-b9ab-76f18ff4f51d,False,1,False
205516,2019-04-24 18:30:50.199,90072729247980,False,NaN,NaN,1,False
157023,2019-04-20 05:36:20.258,135153013040192,False,NaN,NaN,1,False
418098,2019-04-22 18:13:44.805,161514654074162,True,a0231d93-a9be-4052-ba1a-1b0c09c49f6c,True,1,False
198205,2019-04-22 00:09:33.188,168103949904656,False,NaN,NaN,1,False
366926,2019-04-18 16:40:42.839,186034136943920,True,11a503ae-9059-4cc1-bba0-3dcfa8ab635e,NaN,1,False


In [163]:
for i in range(1, len(installsCurated)):
     if (installsCurated.iloc[i,1] == installsCurated.iloc[i-1,1]):
        installsCurated.iloc[i,6] = \
         ((abs((installsCurated.iloc[i,0] - (installsCurated.iloc[i-1 ,0])) / segundo)) < 60)
        if (isNaN(installsCurated.iloc[i,4])== False):
            installsCurated.iloc[i-1, 4] = installsCurated.iloc[i, 4]

### Cantidad de installs antes de suprimir las replicas

In [99]:
len(installsCurated)

481511

In [164]:
installsCurated.head()

,created,ref_hash,implicit,event_uuid,wifi,installed,is_replica
46742,2019-04-21 19:17:47.657,40621409780134,False,NaN,NaN,1,False
398921,2019-04-18 21:11:50.326,41863526108385,True,2f8be0cc-297e-4c9c-a097-1096aa5824b5,False,1,False
249204,2019-04-18 21:11:51.966,41863526108385,False,NaN,False,1,True
112218,2019-04-18 21:17:11.946,41863526108385,False,NaN,False,1,False
112200,2019-04-18 21:17:16.531,41863526108385,True,f07fd6db-2f55-4539-b9ab-76f18ff4f51d,False,1,True


In [169]:
installsCurated.is_replica.value_counts()

False    422872
True      58639
Name: is_replica, dtype: int64

In [167]:
installsCuratedSinRep = installsCurated[installsCurated.is_replica == False]

In [171]:
installsCuratedSinRep.count()

created       422872
ref_hash      422872
implicit      422872
event_uuid     91021
wifi          260780
installed     422872
is_replica    422872
dtype: int64

In [168]:
installsCuratedSinRep.wifi.value_counts()

True     208723
False     52057
Name: wifi, dtype: int64

In [172]:
installsCuratedSinRep.is_replica.value_counts()

False    422872
Name: is_replica, dtype: int64

In [173]:
installsCuratedSinRep = installsCuratedSinRep.drop(columns = ['is_replica', 'implicit'])

In [175]:
installsCuratedSinRep = installsCuratedSinRep.drop(columns = ['event_uuid'])

In [176]:
installsCuratedSinRep.head(2)

,created,ref_hash,wifi,installed
46742,2019-04-21 19:17:47.657,40621409780134,NaN,1
398921,2019-04-18 21:11:50.326,41863526108385,False,1


In [184]:
installsCuratedSinRep.to_csv('installsCuratedSinRep.csv', encoding='utf-8', index=False)

### Con este nuevo set curado, vamos a separar en ventanas de tres dias y despues contar la cantidad de installs por device_id, y el promedio del uso del wifi para el install. Nos quedaremos despues con una fila por device_id

In [177]:
installsVentanaUno = installsCuratedSinRep.loc[(installsCuratedSinRep['created'] < "2019-04-21")]
installsVentanaUno = installsVentanaUno.loc[(installsCuratedSinRep['created'] > "2019-04-18")]

In [178]:
installsVentanaDos = installsCuratedSinRep.loc[(installsCuratedSinRep['created'] < "2019-04-22")]
installsVentanaDos = installsVentanaDos.loc[(installsCuratedSinRep['created'] > "2019-04-19")]

In [179]:
installsVentanaTres = installsCuratedSinRep.loc[(installsCuratedSinRep['created'] < "2019-04-23")]
installsVentanaTres = installsVentanaTres.loc[(installsCuratedSinRep['created'] > "2019-04-20")]

In [180]:
installsVentanaCuatro = installsCuratedSinRep.loc[(installsCuratedSinRep['created'] < "2019-04-24")]
installsVentanaCuatro = installsVentanaCuatro.loc[(installsCuratedSinRep['created'] > "2019-04-21")]

In [181]:
installsVentanaCinco = installsCuratedSinRep.loc[(installsCuratedSinRep['created'] < "2019-04-25")]
installsVentanaCinco = installsVentanaCinco.loc[(installsCuratedSinRep['created'] > "2019-04-22")]

In [182]:
installsVentanaSeis = installsCuratedSinRep.loc[(installsCuratedSinRep['created'] < "2019-04-26")]
installsVentanaSeis = installsVentanaSeis.loc[(installsCuratedSinRep['created'] > "2019-04-23")]

In [183]:
installsVentanaSiete = installsCuratedSinRep.loc[(installsCuratedSinRep['created'] < "2019-04-27")]
installsVentanaSiete = installsVentanaSiete.loc[(installsCuratedSinRep['created'] > "2019-04-24")]

In [185]:
installsVentanaSiete.head(5)

,created,ref_hash,wifi,installed
205516,2019-04-24 18:30:50.199,90072729247980,NaN,1
219372,2019-04-26 22:23:41.836,342614246084071,NaN,1
288979,2019-04-26 21:40:11.227,347785260789835,NaN,1
228155,2019-04-25 15:09:11.886,416301579449694,NaN,1
408449,2019-04-25 17:03:11.040,420449720220692,NaN,1


In [186]:
installsVentanaUno['fechaInicial'] = pd.to_datetime("2019-04-18 00:00:00")
installsVentanaDos['fechaInicial'] = pd.to_datetime("2019-04-19 00:00:00")
installsVentanaTres['fechaInicial'] = pd.to_datetime("2019-04-20 00:00:00")
installsVentanaCuatro['fechaInicial'] = pd.to_datetime("2019-04-21 00:00:00")
installsVentanaCinco['fechaInicial'] = pd.to_datetime("2019-04-22 00:00:00")
installsVentanaSeis['fechaInicial'] = pd.to_datetime("2019-04-23 00:00:00")
installsVentanaSiete['fechaInicial'] = pd.to_datetime("2019-04-24 00:00:00")

In [187]:
installsVentanaUno["created"] = installsVentanaUno["created"] - installsVentanaUno["fechaInicial"]
installsVentanaDos["created"] = installsVentanaDos["created"] - installsVentanaDos["fechaInicial"]
installsVentanaTres["created"] = installsVentanaTres["created"] - installsVentanaTres["fechaInicial"]
installsVentanaCuatro["created"] = installsVentanaCuatro["created"] - installsVentanaCuatro["fechaInicial"]
installsVentanaCinco["created"] = installsVentanaCinco["created"] - installsVentanaCinco["fechaInicial"]
installsVentanaSeis["created"] = installsVentanaSeis["created"] - installsVentanaSeis["fechaInicial"]
installsVentanaSiete["created"] = installsVentanaSiete["created"] - installsVentanaSeis["fechaInicial"]

In [188]:
installsVentanaUno.head()

,created,ref_hash,wifi,installed,fechaInicial
398921,0 days 21:11:50.326000,41863526108385,False,1,2019-04-18
112218,0 days 21:17:11.946000,41863526108385,False,1,2019-04-18
157023,2 days 05:36:20.258000,135153013040192,NaN,1,2019-04-18
366926,0 days 16:40:42.839000,186034136943920,NaN,1,2019-04-18
424433,1 days 02:42:36.508000,365882020742330,False,1,2019-04-18


In [209]:
installsVentanaUno.to_csv('installsVentanaUno.csv', encoding = 'utf_8', index = False)

In [215]:
cantidadInstallsPorIdVentanaUno = installsVentanaUno[['ref_hash', 'installed']].groupby('ref_hash').sum()
cantidadInstallsPorIdVentanaDos = installsVentanaDos[['ref_hash', 'installed']].groupby('ref_hash').sum()
cantidadInstallsPorIdVentanaTres = installsVentanaTres[['ref_hash', 'installed']].groupby('ref_hash').sum()
cantidadInstallsPorIdVentanaCuatro = installsVentanaCuatro[['ref_hash', 'installed']].groupby('ref_hash').sum()
cantidadInstallsPorIdVentanaCinco = installsVentanaCinco[['ref_hash', 'installed']].groupby('ref_hash').sum()
cantidadInstallsPorIdVentanaSeis = installsVentanaSeis[['ref_hash', 'installed']].groupby('ref_hash').sum()
cantidadInstallsPorIdVentanaSiete = installsVentanaSiete[['ref_hash', 'installed']].groupby('ref_hash').sum()

In [217]:
cantidadInstallsPorIdVentanaUno = cantidadInstallsPorIdVentanaUno.stack().reset_index()
cantidadInstallsPorIdVentanaDos = cantidadInstallsPorIdVentanaDos.stack().reset_index()
cantidadInstallsPorIdVentanaTres = cantidadInstallsPorIdVentanaTres.stack().reset_index()
cantidadInstallsPorIdVentanaCuatro = cantidadInstallsPorIdVentanaCuatro.stack().reset_index()
cantidadInstallsPorIdVentanaCinco = cantidadInstallsPorIdVentanaCinco.stack().reset_index()
cantidadInstallsPorIdVentanaSeis = cantidadInstallsPorIdVentanaSeis.stack().reset_index()
cantidadInstallsPorIdVentanaSiete = cantidadInstallsPorIdVentanaSiete.stack().reset_index()

In [218]:
cantidadInstallsPorIdVentanaUno.head(2)

,ref_hash,level_1,0
0,41863526108385,installed,2
1,135153013040192,installed,1


In [219]:
cantidadInstallsPorIdVentanaUno = cantidadInstallsPorIdVentanaUno.drop(columns = ['level_1'])
cantidadInstallsPorIdVentanaDos = cantidadInstallsPorIdVentanaDos.drop(columns = ['level_1'])
cantidadInstallsPorIdVentanaTres = cantidadInstallsPorIdVentanaTres.drop(columns = ['level_1'])
cantidadInstallsPorIdVentanaCuatro = cantidadInstallsPorIdVentanaCuatro.drop(columns = ['level_1'])
cantidadInstallsPorIdVentanaCinco = cantidadInstallsPorIdVentanaCinco.drop(columns = ['level_1'])
cantidadInstallsPorIdVentanaSeis = cantidadInstallsPorIdVentanaSeis.drop(columns = ['level_1'])
cantidadInstallsPorIdVentanaSiete = cantidadInstallsPorIdVentanaSiete.drop(columns = ['level_1'])

In [220]:
cantidadInstallsPorIdVentanaUno.columns = ['ref_hash', 'cantidad_installs']
cantidadInstallsPorIdVentanaDos.columns = ['ref_hash', 'cantidad_installs']
cantidadInstallsPorIdVentanaTres.columns = ['ref_hash', 'cantidad_installs']
cantidadInstallsPorIdVentanaCuatro.columns = ['ref_hash', 'cantidad_installs']
cantidadInstallsPorIdVentanaCinco.columns = ['ref_hash', 'cantidad_installs']
cantidadInstallsPorIdVentanaSeis.columns = ['ref_hash', 'cantidad_installs']
cantidadInstallsPorIdVentanaSiete.columns = ['ref_hash', 'cantidad_installs']

In [221]:
cantidadInstallsPorIdVentanaSiete.head(2)

,ref_hash,cantidad_installs
0,90072729247980,1
1,342614246084071,1


In [224]:
installsVentanaUno['wifi2'] = np.nan

In [225]:
installsVentanaDos['wifi2'] = np.nan
installsVentanaTres['wifi2'] = np.nan
installsVentanaCuatro['wifi2'] = np.nan
installsVentanaCinco['wifi2'] = np.nan
installsVentanaSeis['wifi2'] = np.nan
installsVentanaSiete['wifi2'] = np.nan

In [226]:
installsVentanaUno.loc[installsVentanaUno.wifi == True, 'wifi2'] = 1
installsVentanaUno.loc[installsVentanaUno.wifi == False, 'wifi2'] = 0

In [229]:
installsVentanaDos.loc[installsVentanaDos.wifi == True, 'wifi2'] = 1
installsVentanaDos.loc[installsVentanaDos.wifi == False, 'wifi2'] = 0
installsVentanaTres.loc[installsVentanaTres.wifi == True, 'wifi2'] = 1
installsVentanaTres.loc[installsVentanaTres.wifi == False, 'wifi2'] = 0
installsVentanaCuatro.loc[installsVentanaCuatro.wifi == True, 'wifi2'] = 1
installsVentanaCuatro.loc[installsVentanaCuatro.wifi == False, 'wifi2'] = 0
installsVentanaCinco.loc[installsVentanaCinco.wifi == True, 'wifi2'] = 1
installsVentanaCinco.loc[installsVentanaCinco.wifi == False, 'wifi2'] = 0
installsVentanaSeis.loc[installsVentanaSeis.wifi == True, 'wifi2'] = 1
installsVentanaSeis.loc[installsVentanaSeis.wifi == False, 'wifi2'] = 0
installsVentanaSiete.loc[installsVentanaSiete.wifi == True, 'wifi2'] = 1
installsVentanaSiete.loc[installsVentanaSiete.wifi == False, 'wifi2'] = 0

In [227]:
installsWifiPromedioPorIdVentanaUno = installsVentanaUno[['ref_hash', 'wifi2']].groupby('ref_hash').mean()

In [230]:
installsWifiPromedioPorIdVentanaDos = installsVentanaDos[['ref_hash', 'wifi2']].groupby('ref_hash').mean()
installsWifiPromedioPorIdVentanaTres = installsVentanaTres[['ref_hash', 'wifi2']].groupby('ref_hash').mean()
installsWifiPromedioPorIdVentanaCuatro = installsVentanaCuatro[['ref_hash', 'wifi2']].groupby('ref_hash').mean()
installsWifiPromedioPorIdVentanaCinco = installsVentanaCinco[['ref_hash', 'wifi2']].groupby('ref_hash').mean()
installsWifiPromedioPorIdVentanaSeis = installsVentanaSeis[['ref_hash', 'wifi2']].groupby('ref_hash').mean()
installsWifiPromedioPorIdVentanaSiete = installsVentanaSiete[['ref_hash', 'wifi2']].groupby('ref_hash').mean()

In [228]:
installsWifiPromedioPorIdVentanaUno.head(2)

,wifi2
ref_hash,
41863526108385,0.0
135153013040192,NaN


In [231]:
installsWifiPromedioPorIdVentanaUno = installsWifiPromedioPorIdVentanaUno.stack().reset_index()
installsWifiPromedioPorIdVentanaDos = installsWifiPromedioPorIdVentanaDos.stack().reset_index()
installsWifiPromedioPorIdVentanaTres = installsWifiPromedioPorIdVentanaTres.stack().reset_index()
installsWifiPromedioPorIdVentanaCuatro = installsWifiPromedioPorIdVentanaCuatro.stack().reset_index()
installsWifiPromedioPorIdVentanaCinco = installsWifiPromedioPorIdVentanaCinco.stack().reset_index()
installsWifiPromedioPorIdVentanaSeis = installsWifiPromedioPorIdVentanaSeis.stack().reset_index()
installsWifiPromedioPorIdVentanaSiete = installsWifiPromedioPorIdVentanaSiete.stack().reset_index()

In [232]:
installsWifiPromedioPorIdVentanaUno = installsWifiPromedioPorIdVentanaUno.drop(columns = ['level_1'])
installsWifiPromedioPorIdVentanaDos = installsWifiPromedioPorIdVentanaDos.drop(columns = ['level_1'])
installsWifiPromedioPorIdVentanaTres = installsWifiPromedioPorIdVentanaTres.drop(columns = ['level_1'])
installsWifiPromedioPorIdVentanaCuatro = installsWifiPromedioPorIdVentanaCuatro.drop(columns = ['level_1'])
installsWifiPromedioPorIdVentanaCinco = installsWifiPromedioPorIdVentanaCinco.drop(columns = ['level_1'])
installsWifiPromedioPorIdVentanaSeis = installsWifiPromedioPorIdVentanaSeis.drop(columns = ['level_1'])
installsWifiPromedioPorIdVentanaSiete = installsWifiPromedioPorIdVentanaSiete.drop(columns = ['level_1'])

In [233]:
installsWifiPromedioPorIdVentanaUno.columns = ['ref_hash', 'wifi_promedio_inst']
installsWifiPromedioPorIdVentanaDos.columns = ['ref_hash', 'wifi_promedio_inst']
installsWifiPromedioPorIdVentanaTres.columns = ['ref_hash', 'wifi_promedio_inst']
installsWifiPromedioPorIdVentanaCuatro.columns = ['ref_hash', 'wifi_promedio_inst']
installsWifiPromedioPorIdVentanaCinco.columns = ['ref_hash', 'wifi_promedio_inst']
installsWifiPromedioPorIdVentanaSeis.columns = ['ref_hash', 'wifi_promedio_inst']
installsWifiPromedioPorIdVentanaSiete.columns = ['ref_hash', 'wifi_promedio_inst']

In [234]:
installsWifiPromedioPorIdVentanaUno.head(3)

,ref_hash,wifi_promedio_inst
0,41863526108385,0.0
1,365882020742330,0.0
2,519199987760489,1.0


In [237]:
installsVentanaUno = pd.merge(installsVentanaUno, installsWifiPromedioPorIdVentanaUno, on = 'ref_hash', how = 'left')
installsVentanaDos = pd.merge(installsVentanaDos, installsWifiPromedioPorIdVentanaDos, on = 'ref_hash', how = 'left')
installsVentanaTres = pd.merge(installsVentanaTres, installsWifiPromedioPorIdVentanaTres, on = 'ref_hash', how = 'left')
installsVentanaCuatro = pd.merge(installsVentanaCuatro, installsWifiPromedioPorIdVentanaCuatro, on = 'ref_hash', how = 'left')
installsVentanaCinco = pd.merge(installsVentanaCinco, installsWifiPromedioPorIdVentanaCinco, on = 'ref_hash', how = 'left')
installsVentanaSeis = pd.merge(installsVentanaSeis, installsWifiPromedioPorIdVentanaSeis, on = 'ref_hash', how = 'left')
installsVentanaSiete = pd.merge(installsVentanaSiete, installsWifiPromedioPorIdVentanaSiete, on = 'ref_hash', how = 'left')


In [240]:
installsVentanaUno = pd.merge(installsVentanaUno, cantidadInstallsPorIdVentanaUno, on = 'ref_hash', how = 'left')
installsVentanaDos = pd.merge(installsVentanaDos, cantidadInstallsPorIdVentanaDos, on = 'ref_hash', how = 'left')
installsVentanaTres = pd.merge(installsVentanaTres, cantidadInstallsPorIdVentanaTres, on = 'ref_hash', how = 'left')
installsVentanaCuatro = pd.merge(installsVentanaCuatro, cantidadInstallsPorIdVentanaCuatro, on = 'ref_hash', how = 'left')
installsVentanaCinco = pd.merge(installsVentanaCinco, cantidadInstallsPorIdVentanaCinco, on = 'ref_hash', how = 'left')
installsVentanaSeis = pd.merge(installsVentanaSeis, cantidadInstallsPorIdVentanaSeis, on = 'ref_hash', how = 'left')
installsVentanaSiete = pd.merge(installsVentanaSiete, cantidadInstallsPorIdVentanaSiete, on = 'ref_hash', how = 'left')


In [242]:
installsVentanaUno.head(2)

,created,ref_hash,wifi,installed,fechaInicial,wifi2,wifi_promedio_inst,cantidad_installs
0,21:11:50.326000,41863526108385,False,1,2019-04-18,0.0,0.0,2
1,21:17:11.946000,41863526108385,False,1,2019-04-18,0.0,0.0,2


In [243]:
installsIdUnicoVentanaUno = installsVentanaUno[['created', 'ref_hash', 'wifi_promedio_inst', 'cantidad_installs']]
installsIdUnicoVentanaDos = installsVentanaDos[['created', 'ref_hash', 'wifi_promedio_inst', 'cantidad_installs']]
installsIdUnicoVentanaTres = installsVentanaTres[['created', 'ref_hash', 'wifi_promedio_inst', 'cantidad_installs']]
installsIdUnicoVentanaCuatro = installsVentanaCuatro[['created', 'ref_hash', 'wifi_promedio_inst', 'cantidad_installs']]
installsIdUnicoVentanaCinco = installsVentanaCinco[['created', 'ref_hash', 'wifi_promedio_inst', 'cantidad_installs']]
installsIdUnicoVentanaSeis = installsVentanaSeis[['created', 'ref_hash', 'wifi_promedio_inst', 'cantidad_installs']]
installsIdUnicoVentanaSiete = installsVentanaSiete[['created', 'ref_hash', 'wifi_promedio_inst', 'cantidad_installs']]

In [245]:
installsIdUnicoVentanaUno = installsIdUnicoVentanaUno.drop_duplicates(subset = 'ref_hash')
installsIdUnicoVentanaDos = installsIdUnicoVentanaDos.drop_duplicates(subset = 'ref_hash')
installsIdUnicoVentanaTres = installsIdUnicoVentanaTres.drop_duplicates(subset = 'ref_hash')
installsIdUnicoVentanaCuatro = installsIdUnicoVentanaCuatro.drop_duplicates(subset = 'ref_hash')
installsIdUnicoVentanaCinco = installsIdUnicoVentanaCinco.drop_duplicates(subset = 'ref_hash')
installsIdUnicoVentanaSeis = installsIdUnicoVentanaSeis.drop_duplicates(subset = 'ref_hash')
installsIdUnicoVentanaSiete = installsIdUnicoVentanaSiete.drop_duplicates(subset = 'ref_hash')

In [297]:
installsIdUnicoVentanaUno["created"] = installsIdUnicoVentanaUno["created"].dt.total_seconds()
installsIdUnicoVentanaDos["created"] = installsIdUnicoVentanaDos["created"].dt.total_seconds()
installsIdUnicoVentanaTres["created"] = installsIdUnicoVentanaTres["created"].dt.total_seconds()
installsIdUnicoVentanaCuatro["created"] = installsIdUnicoVentanaCuatro["created"].dt.total_seconds()
installsIdUnicoVentanaCinco["created"] = installsIdUnicoVentanaCinco["created"].dt.total_seconds()
installsIdUnicoVentanaSeis["created"] = installsIdUnicoVentanaSeis["created"].dt.total_seconds()
installsIdUnicoVentanaSiete["created"] = installsIdUnicoVentanaSiete["created"].dt.total_seconds()

In [246]:
installsIdUnicoVentanaUno.head(10)

,created,ref_hash,wifi_promedio_inst,cantidad_installs
0,0 days 21:11:50.326000,41863526108385,0.0,2
2,2 days 05:36:20.258000,135153013040192,NaN,1
3,0 days 16:40:42.839000,186034136943920,NaN,1
4,1 days 02:42:36.508000,365882020742330,0.0,1
5,1 days 01:47:40.475000,519199987760489,1.0,1
6,0 days 20:51:02.237000,530786270564316,0.0,1
7,0 days 18:10:00.392000,558877640599287,1.0,1
8,0 days 20:31:37.181000,655267966876774,0.0,1
9,1 days 06:30:37.123000,848739744907305,1.0,1
10,2 days 13:14:07.027000,918552236966790,NaN,1


In [248]:
eventsCurated.head(2)

,index,date,event_id,ref_hash,application_id,attributed,device_city,event_uuid,carrier,kind,device_os,wifi,connection_type,device_language
0,2130678,2019-04-20 01:42:49.120,0,5857744372586891366,210,False,NaN,5b506964-5f47-4b28-a8c2-8a92d6c23379,NaN,5.882882e+18,NaN,False,NaN,3.301378e+18
1,2130680,2019-04-20 01:42:49.340,1,7642521036780133571,210,False,NaN,f1fb9d15-1a7b-4116-8d3b-c4c403e197e2,NaN,4.017674e+18,NaN,False,NaN,NaN


In [249]:
eventsCurated = eventsCurated[['date', 'ref_hash']]

In [250]:
eventsCurated.head(2)

,date,ref_hash
0,2019-04-20 01:42:49.120,5857744372586891366
1,2019-04-20 01:42:49.340,7642521036780133571


## Selección de las ventanas de tests de tres días en _Events_

In [252]:
eventsVentanaUno = eventsCurated.loc[(eventsCurated["date"] < "2019-04-21")]
eventsVentanaUno = eventsVentanaUno.loc[(eventsCurated["date"] > "2019-04-18")]

In [253]:
eventsVentanaDos = eventsCurated.loc[(eventsCurated["date"] < "2019-04-22")]
eventsVentanaDos = eventsVentanaDos.loc[(eventsCurated["date"] > "2019-04-19")]

In [254]:
eventsVentanaTres = eventsCurated.loc[(eventsCurated["date"] < "2019-04-23")]
eventsVentanaTres = eventsVentanaTres.loc[(eventsCurated["date"] > "2019-04-20")]

In [255]:
eventsVentanaCuatro = eventsCurated.loc[(eventsCurated["date"] < "2019-04-24")]
eventsVentanaCuatro = eventsVentanaCuatro.loc[(eventsCurated["date"] > "2019-04-21")]

In [256]:
eventsVentanaCinco = eventsCurated.loc[(eventsCurated["date"] < "2019-04-25")]
eventsVentanaCinco = eventsVentanaCinco.loc[(eventsCurated["date"] > "2019-04-22")]

In [257]:
eventsVentanaSeis = eventsCurated.loc[(eventsCurated["date"] < "2019-04-26")]
eventsVentanaSeis = eventsVentanaSeis.loc[(eventsCurated["date"] > "2019-04-23")]

In [258]:
eventsVentanaSiete = eventsCurated.loc[(eventsCurated["date"] < "2019-04-27")]
eventsVentanaSiete = eventsVentanaSiete.loc[(eventsCurated["date"] > "2019-04-24")]

### En todas las ventanas elegidas, vamos a agregar una columna de fecha inicial para poder basar los tiempos en base a eso. Despues podremos restar el tiempo inicial al tiempo absoluto para hacer modelos en segundos pasados.

In [259]:
eventsVentanaUno['fechaInicial'] = pd.to_datetime("2019-04-18 00:00:00")
eventsVentanaDos['fechaInicial'] = pd.to_datetime("2019-04-19 00:00:00")
eventsVentanaTres['fechaInicial'] = pd.to_datetime("2019-04-20 00:00:00")
eventsVentanaCuatro['fechaInicial'] = pd.to_datetime("2019-04-21 00:00:00")
eventsVentanaCinco['fechaInicial'] = pd.to_datetime("2019-04-22 00:00:00")
eventsVentanaSeis['fechaInicial'] = pd.to_datetime("2019-04-23 00:00:00")
eventsVentanaSiete['fechaInicial'] = pd.to_datetime("2019-04-24 00:00:00")

In [260]:
eventsVentanaUno.head()

,date,ref_hash,fechaInicial
0,2019-04-20 01:42:49.120,5857744372586891366,2019-04-18
1,2019-04-20 01:42:49.340,7642521036780133571,2019-04-18
2,2019-04-20 01:42:49.365,2548841562898283198,2019-04-18
3,2019-04-20 01:42:51.438,609402887625919085,2019-04-18
4,2019-04-20 01:42:51.838,9114651763556439823,2019-04-18


### Convertimos los objects de _date_ a elementos de tipo _datetime_ para poder calcular diferencias de tiempo y poder utilisar modelos de Lifelines.

In [261]:
eventsVentanaUno.loc[:,'date'] = pd.to_datetime(eventsVentanaUno['date'], errors = 'coerce')
eventsVentanaDos.loc[:,'date'] = pd.to_datetime(eventsVentanaDos['date'], errors = 'coerce')
eventsVentanaTres.loc[:,'date'] = pd.to_datetime(eventsVentanaTres['date'], errors = 'coerce')
eventsVentanaCuatro.loc[:,'date'] = pd.to_datetime(eventsVentanaCuatro['date'], errors = 'coerce')
eventsVentanaCinco.loc[:,'date'] = pd.to_datetime(eventsVentanaCinco['date'], errors = 'coerce')
eventsVentanaSeis.loc[:,'date'] = pd.to_datetime(eventsVentanaSeis['date'], errors = 'coerce')
eventsVentanaSiete.loc[:,'date'] = pd.to_datetime(eventsVentanaSiete['date'], errors = 'coerce')

### Sacamos la diferencia entre el tiempo de la ventana y el tiempo inicial de la ventana

In [262]:
eventsVentanaUno["date"] = eventsVentanaUno["date"] - eventsVentanaUno["fechaInicial"]
eventsVentanaDos["date"] = eventsVentanaDos["date"] - eventsVentanaDos["fechaInicial"]
eventsVentanaTres["date"] = eventsVentanaTres["date"] - eventsVentanaTres["fechaInicial"]
eventsVentanaCuatro["date"] = eventsVentanaCuatro["date"] - eventsVentanaCuatro["fechaInicial"]
eventsVentanaCinco["date"] = eventsVentanaCinco["date"] - eventsVentanaCinco["fechaInicial"]
eventsVentanaSeis["date"] = eventsVentanaSeis["date"] - eventsVentanaSeis["fechaInicial"]
eventsVentanaSiete["date"] = eventsVentanaSiete["date"] - eventsVentanaSiete["fechaInicial"]

In [263]:
eventsVentanaUno.head(2)

,date,ref_hash,fechaInicial
0,2 days 01:42:49.120000,5857744372586891366,2019-04-18
1,2 days 01:42:49.340000,7642521036780133571,2019-04-18


### Queremos resultados en segundos

In [264]:
eventsVentanaUno["date"] = eventsVentanaUno["date"].dt.total_seconds()
eventsVentanaDos["date"] = eventsVentanaDos["date"].dt.total_seconds()
eventsVentanaTres["date"] = eventsVentanaTres["date"].dt.total_seconds()
eventsVentanaCuatro["date"] = eventsVentanaCuatro["date"].dt.total_seconds()
eventsVentanaCinco["date"] = eventsVentanaCinco["date"].dt.total_seconds()
eventsVentanaSeis["date"] = eventsVentanaSeis["date"].dt.total_seconds()
eventsVentanaSiete["date"] = eventsVentanaSiete["date"].dt.total_seconds()

Agregamos una columna _sum_ para poder contar la cantidad de veces que aparece un device en una ventana de tres días.

In [266]:
eventsVentanaUno["sum"] = 1
eventsVentanaDos["sum"] = 1
eventsVentanaTres["sum"] = 1
eventsVentanaCuatro["sum"] = 1
eventsVentanaCinco["sum"] = 1
eventsVentanaSeis["sum"] = 1
eventsVentanaSiete["sum"] = 1

Contamos la cantidad de auctions en la ventana de 3 días para incluirla en la ventana siguiente como parametro.

In [267]:
eventsPorIdVentanaUno = eventsVentanaUno[['ref_hash','sum']].groupby('ref_hash').sum()
eventsPorIdVentanaDos = eventsVentanaDos[['ref_hash','sum']].groupby('ref_hash').sum()
eventsPorIdVentanaTres = eventsVentanaTres[['ref_hash','sum']].groupby('ref_hash').sum()
eventsPorIdVentanaCuatro = eventsVentanaCuatro[['ref_hash','sum']].groupby('ref_hash').sum()
eventsPorIdVentanaCinco = eventsVentanaCinco[['ref_hash','sum']].groupby('ref_hash').sum()
eventsPorIdVentanaSeis = eventsVentanaSeis[['ref_hash','sum']].groupby('ref_hash').sum()
eventsPorIdVentanaSiete = eventsVentanaSiete[['ref_hash','sum']].groupby('ref_hash').sum()

In [268]:
eventsPorIdVentanaSeis.head(2)

,sum
ref_hash,
40621409780134,3
69039685746313,6


In [269]:
eventsPorIdVentanaUno = eventsPorIdVentanaUno.stack().reset_index()
eventsPorIdVentanaDos = eventsPorIdVentanaDos.stack().reset_index()
eventsPorIdVentanaTres = eventsPorIdVentanaTres.stack().reset_index()
eventsPorIdVentanaCuatro = eventsPorIdVentanaCuatro.stack().reset_index()
eventsPorIdVentanaCinco = eventsPorIdVentanaCinco.stack().reset_index()
eventsPorIdVentanaSeis = eventsPorIdVentanaSeis.stack().reset_index()
eventsPorIdVentanaSiete = eventsPorIdVentanaSiete.stack().reset_index()

In [270]:
eventsPorIdVentanaUno = eventsPorIdVentanaUno.drop(columns = ['level_1'])
eventsPorIdVentanaDos = eventsPorIdVentanaDos.drop(columns = ['level_1'])
eventsPorIdVentanaTres = eventsPorIdVentanaTres.drop(columns = ['level_1'])
eventsPorIdVentanaCuatro = eventsPorIdVentanaCuatro.drop(columns = ['level_1'])
eventsPorIdVentanaCinco = eventsPorIdVentanaCinco.drop(columns = ['level_1'])
eventsPorIdVentanaSeis = eventsPorIdVentanaSeis.drop(columns = ['level_1'])
eventsPorIdVentanaSiete = eventsPorIdVentanaSiete.drop(columns = ['level_1'])

In [273]:
eventsPorIdVentanaUno.columns = ['ref_hash', 'cantidad_events']
eventsPorIdVentanaDos.columns = ['ref_hash', 'cantidad_events']
eventsPorIdVentanaTres.columns = ['ref_hash', 'cantidad_events']
eventsPorIdVentanaCuatro.columns = ['ref_hash', 'cantidad_events']
eventsPorIdVentanaCinco.columns = ['ref_hash', 'cantidad_events']
eventsPorIdVentanaSeis.columns = ['ref_hash', 'cantidad_events']
eventsPorIdVentanaSiete.columns = ['ref_hash', 'cantidad_events']

In [274]:
eventsPorIdVentanaUno.head(2)

,ref_hash,cantidad_events
0,41863526108385,88
1,161514654074162,8


In [283]:
eventsVentanaUno = pd.merge(eventsVentanaUno, eventsPorIdVentanaUno, on ='ref_hash', how = 'left')
eventsVentanaDos = pd.merge(eventsVentanaDos, eventsPorIdVentanaDos, on ='ref_hash', how = 'left')
eventsVentanaTres = pd.merge(eventsVentanaTres, eventsPorIdVentanaTres, on ='ref_hash', how = 'left')
eventsVentanaCuatro = pd.merge(eventsVentanaCuatro, eventsPorIdVentanaCuatro, on ='ref_hash', how = 'left')
eventsVentanaCinco = pd.merge(eventsVentanaCinco, eventsPorIdVentanaCinco, on ='ref_hash', how = 'left')
eventsVentanaSeis = pd.merge(eventsVentanaSeis, eventsPorIdVentanaSeis, on ='ref_hash', how = 'left')
eventsVentanaSiete = pd.merge(eventsVentanaSiete, eventsPorIdVentanaSiete, on ='ref_hash', how = 'left')


In [286]:
eventsVentanaSiete.head(10)

,date,ref_hash,fechaInicial,sum,cantidad_events
0,163534.650,809939361959643854,2019-04-24,1,40
1,163533.165,2704332589081852700,2019-04-24,1,546
2,163533.097,2704332589081852700,2019-04-24,1,546
3,163534.227,6376777580200607439,2019-04-24,1,459
4,163050.501,2602532777370559745,2019-04-24,1,62
5,162446.249,1392327325375134465,2019-04-24,1,94
6,163054.553,6452440169362283903,2019-04-24,1,36
7,163054.490,3346836796007974949,2019-04-24,1,55
8,162457.989,2576380917892592388,2019-04-24,1,29
9,163054.447,1392327325375134465,2019-04-24,1,94


### Vamos a dejar los duplicates.

### Para hacer eso, ordenaremos por device_id y despues por fecha

In [287]:
eventsVentanaUno = eventsVentanaUno.sort_values(['ref_hash', 'date'], ascending = [True, True])
eventsVentanaDos = eventsVentanaDos.sort_values(['ref_hash', 'date'], ascending = [True, True])
eventsVentanaTres = eventsVentanaTres.sort_values(['ref_hash', 'date'], ascending = [True, True])
eventsVentanaCuatro = eventsVentanaCuatro.sort_values(['ref_hash', 'date'], ascending = [True, True])
eventsVentanaCinco = eventsVentanaCinco.sort_values(['ref_hash', 'date'], ascending = [True, True])
eventsVentanaSeis = eventsVentanaSeis.sort_values(['ref_hash', 'date'], ascending = [True, True])
eventsVentanaSiete = eventsVentanaSiete.sort_values(['ref_hash', 'date'], ascending = [True, True])

In [289]:
eventsVentanaUno.head()

,date,ref_hash,fechaInicial,sum,cantidad_events
269120,76310.326,41863526108385,2019-04-18,1,88
933324,76385.063,41863526108385,2019-04-18,1,88
876951,76636.531,41863526108385,2019-04-18,1,88
876939,76641.831,41863526108385,2019-04-18,1,88
1044023,76678.316,41863526108385,2019-04-18,1,88


In [291]:
eventsVentanaUnoRegressionSet = eventsVentanaUno.drop_duplicates(subset = 'ref_hash')
eventsVentanaDosRegressionSet = eventsVentanaDos.drop_duplicates(subset = 'ref_hash')
eventsVentanaTresRegressionSet = eventsVentanaTres.drop_duplicates(subset = 'ref_hash')
eventsVentanaCuatroRegressionSet = eventsVentanaCuatro.drop_duplicates(subset = 'ref_hash')
eventsVentanaCincoRegressionSet = eventsVentanaCinco.drop_duplicates(subset = 'ref_hash')
eventsVentanaSeisRegressionSet = eventsVentanaSeis.drop_duplicates(subset = 'ref_hash')
eventsVentanaSieteRegressionSet = eventsVentanaSiete.drop_duplicates(subset = 'ref_hash')

In [293]:
eventsVentanaUnoRegressionSet = eventsVentanaUnoRegressionSet[['ref_hash', 'cantidad_events']]
eventsVentanaDosRegressionSet = eventsVentanaDosRegressionSet[['ref_hash', 'cantidad_events']]
eventsVentanaTresRegressionSet = eventsVentanaTresRegressionSet[['ref_hash', 'cantidad_events']]
eventsVentanaCuatroRegressionSet = eventsVentanaCuatroRegressionSet[['ref_hash', 'cantidad_events']]
eventsVentanaCincoRegressionSet = eventsVentanaCincoRegressionSet[['ref_hash', 'cantidad_events']]
eventsVentanaSeisRegressionSet = eventsVentanaSeisRegressionSet[['ref_hash', 'cantidad_events']]
eventsVentanaSieteRegressionSet = eventsVentanaSieteRegressionSet[['ref_hash', 'cantidad_events']]

In [294]:
eventsVentanaUnoRegressionSet.head()

,ref_hash,cantidad_events
269120,41863526108385,88
1209992,161514654074162,8
326573,186034136943920,13
1072019,360710529886978,3
1227073,365882020742330,57


In [474]:
eventsVentanaUnoRegressionSet.to_csv('events_set1.csv', encoding = 'utf_8', index=False)
eventsVentanaDosRegressionSet.to_csv('events_set2.csv', encoding = 'utf_8', index=False)
eventsVentanaTresRegressionSet.to_csv('events_set3.csv', encoding = 'utf_8', index=False)
eventsVentanaCuatroRegressionSet.to_csv('events_set4.csv', encoding = 'utf_8', index=False)
eventsVentanaCincoRegressionSet.to_csv('events_set5.csv', encoding = 'utf_8', index=False)
eventsVentanaSeisRegressionSet.to_csv('events_set6.csv', encoding = 'utf_8', index=False)
eventsVentanaSieteRegressionSet.to_csv('events_set7.csv', encoding = 'utf_8', index=False)

In [475]:
installsIdUnicoVentanaUno.to_csv('installs_set1.csv', encoding = 'utf_8', index=False)
installsIdUnicoVentanaDos.to_csv('installs_set2.csv', encoding = 'utf_8', index=False)
installsIdUnicoVentanaTres.to_csv('installs_set3.csv', encoding = 'utf_8', index=False)
installsIdUnicoVentanaCuatro.to_csv('installs_set4.csv', encoding = 'utf_8', index=False)
installsIdUnicoVentanaCinco.to_csv('installs_set5.csv', encoding = 'utf_8', index=False)
installsIdUnicoVentanaSeis.to_csv('installs_set6.csv', encoding = 'utf_8', index=False)
installsIdUnicoVentanaSiete.to_csv('installs_set7.csv', encoding = 'utf_8', index=False)

### Ahora mergeamos events y Installs on ref_hash

In [298]:
eventsInstallsVentUnoRegressionSet = pd.merge(eventsVentanaUnoRegressionSet, installsIdUnicoVentanaUno, on='ref_hash', how = 'outer')
eventsInstallsVentDosRegressionSet = pd.merge(eventsVentanaDosRegressionSet, installsIdUnicoVentanaDos, on='ref_hash', how = 'outer')
eventsInstallsVentTresRegressionSet = pd.merge(eventsVentanaTresRegressionSet, installsIdUnicoVentanaTres, on='ref_hash', how = 'outer')
eventsInstallsVentCuatroRegressionSet = pd.merge(eventsVentanaCuatroRegressionSet, installsIdUnicoVentanaCuatro, on='ref_hash', how = 'outer')
eventsInstallsVentCincoRegressionSet = pd.merge(eventsVentanaCincoRegressionSet, installsIdUnicoVentanaCinco, on='ref_hash', how = 'outer')
eventsInstallsVentSeisRegressionSet = pd.merge(eventsVentanaSeisRegressionSet, installsIdUnicoVentanaSeis, on='ref_hash', how = 'outer')
eventsInstallsVentSieteRegressionSet = pd.merge(eventsVentanaSieteRegressionSet, installsIdUnicoVentanaSiete, on='ref_hash', how = 'outer')


In [299]:
eventsInstallsVentUnoRegressionSet.head()

,ref_hash,cantidad_events,created,wifi_promedio_inst,cantidad_installs
0,41863526108385,88.0,76310.326,0.0,2.0
1,161514654074162,8.0,NaN,NaN,NaN
2,186034136943920,13.0,60042.839,NaN,1.0
3,360710529886978,3.0,NaN,NaN,NaN
4,365882020742330,57.0,96156.508,0.0,1.0


In [302]:
eventsInstallsVentUnoRegressionSet[['wifi_promedio_inst', 'cantidad_installs']] = \
eventsInstallsVentUnoRegressionSet[['wifi_promedio_inst', 'cantidad_installs']].fillna(value = 0)
eventsInstallsVentDosRegressionSet[['wifi_promedio_inst', 'cantidad_installs']] = \
eventsInstallsVentDosRegressionSet[['wifi_promedio_inst', 'cantidad_installs']].fillna(value = 0)
eventsInstallsVentTresRegressionSet[['wifi_promedio_inst', 'cantidad_installs']] = \
eventsInstallsVentTresRegressionSet[['wifi_promedio_inst', 'cantidad_installs']].fillna(value = 0)
eventsInstallsVentCuatroRegressionSet[['wifi_promedio_inst', 'cantidad_installs']] = \
eventsInstallsVentCuatroRegressionSet[['wifi_promedio_inst', 'cantidad_installs']].fillna(value = 0)
eventsInstallsVentCincoRegressionSet[['wifi_promedio_inst', 'cantidad_installs']] = \
eventsInstallsVentCincoRegressionSet[['wifi_promedio_inst', 'cantidad_installs']].fillna(value = 0)
eventsInstallsVentSeisRegressionSet[['wifi_promedio_inst', 'cantidad_installs']] = \
eventsInstallsVentSeisRegressionSet[['wifi_promedio_inst', 'cantidad_installs']].fillna(value = 0)
eventsInstallsVentSieteRegressionSet[['wifi_promedio_inst', 'cantidad_installs']] = \
eventsInstallsVentSieteRegressionSet[['wifi_promedio_inst', 'cantidad_installs']].fillna(value = 0)

In [309]:
eventsInstallsVentUnoRegressionSet['cantidad_events'] = \
eventsInstallsVentUnoRegressionSet['cantidad_events'].fillna(eventsInstallsVentUnoRegressionSet['cantidad_installs'])

In [311]:
eventsInstallsVentDosRegressionSet['cantidad_events'] = \
eventsInstallsVentDosRegressionSet['cantidad_events'].fillna(eventsInstallsVentDosRegressionSet['cantidad_installs'])
eventsInstallsVentTresRegressionSet['cantidad_events'] = \
eventsInstallsVentTresRegressionSet['cantidad_events'].fillna(eventsInstallsVentTresRegressionSet['cantidad_installs'])
eventsInstallsVentCuatroRegressionSet['cantidad_events'] = \
eventsInstallsVentCuatroRegressionSet['cantidad_events'].fillna(eventsInstallsVentCuatroRegressionSet['cantidad_installs'])
eventsInstallsVentCincoRegressionSet['cantidad_events'] = \
eventsInstallsVentCincoRegressionSet['cantidad_events'].fillna(eventsInstallsVentCincoRegressionSet['cantidad_installs'])
eventsInstallsVentSeisRegressionSet['cantidad_events'] = \
eventsInstallsVentSeisRegressionSet['cantidad_events'].fillna(eventsInstallsVentSeisRegressionSet['cantidad_installs'])
eventsInstallsVentSieteRegressionSet['cantidad_events'] = \
eventsInstallsVentSieteRegressionSet['cantidad_events'].fillna(eventsInstallsVentSieteRegressionSet['cantidad_installs'])


In [313]:
eventsInstallsVentUnoRegressionSet['created'] = \
eventsInstallsVentUnoRegressionSet['created'].fillna(value = 259200)
eventsInstallsVentDosRegressionSet['created'] = \
eventsInstallsVentDosRegressionSet['created'].fillna(value = 259200)
eventsInstallsVentTresRegressionSet['created'] = \
eventsInstallsVentTresRegressionSet['created'].fillna(value = 259200)
eventsInstallsVentCuatroRegressionSet['created'] = \
eventsInstallsVentCuatroRegressionSet['created'].fillna(value = 259200)
eventsInstallsVentCincoRegressionSet['created'] = \
eventsInstallsVentCincoRegressionSet['created'].fillna(value = 259200)
eventsInstallsVentSeisRegressionSet['created'] = \
eventsInstallsVentSeisRegressionSet['created'].fillna(value = 259200)
eventsInstallsVentSieteRegressionSet['created'] = \
eventsInstallsVentSieteRegressionSet['created'].fillna(value = 259200)


In [315]:
eventsInstallsVentUnoRegressionSet.head()

,ref_hash,cantidad_events,created,wifi_promedio_inst,cantidad_installs
0,41863526108385,88.0,76310.326,0.0,2.0
1,161514654074162,8.0,259200.000,0.0,0.0
2,186034136943920,13.0,60042.839,0.0,1.0
3,360710529886978,3.0,259200.000,0.0,0.0
4,365882020742330,57.0,96156.508,0.0,1.0


In [316]:
eventsInstallsVentUnoRegressionSet.columns = ['ref_hash','events_v1', 'date_v1', 'wifi_v1', 'installs_v1']
eventsInstallsVentDosRegressionSet.columns = ['ref_hash','events_v2', 'date_v2', 'wifi_v2', 'installs_v2']
eventsInstallsVentTresRegressionSet.columns = ['ref_hash','events_v3', 'date_v3', 'wifi_v3', 'installs_v3']
eventsInstallsVentCuatroRegressionSet.columns = ['ref_hash','events_v4', 'date_v4', 'wifi_v4', 'installs_v4']
eventsInstallsVentCincoRegressionSet.columns = ['ref_hash','events_v5', 'date_v5', 'wifi_v5', 'installs_v5']
eventsInstallsVentSeisRegressionSet.columns = ['ref_hash','events_v6', 'date_v6', 'wifi_v6', 'installs_v6']
eventsInstallsVentSieteRegressionSet.columns = ['ref_hash','events_v7', 'date_v7', 'wifi_v7', 'installs_v7']

In [476]:
eventsInstallsVentUnoRegressionSet.to_csv('eventsInstalls_set1.csv', encoding = 'utf_8', index=False)
eventsInstallsVentDosRegressionSet.to_csv('eventsInstalls_set2.csv', encoding = 'utf_8', index=False)
eventsInstallsVentTresRegressionSet.to_csv('eventsInstalls_set3.csv', encoding = 'utf_8', index=False)
eventsInstallsVentCuatroRegressionSet.to_csv('eventsInstalls_set4.csv', encoding = 'utf_8', index=False)
eventsInstallsVentCincoRegressionSet.to_csv('eventsInstalls_set5.csv', encoding = 'utf_8', index=False)
eventsInstallsVentSeisRegressionSet.to_csv('eventsInstalls_set6.csv', encoding = 'utf_8', index=False)
eventsInstallsVentSieteRegressionSet.to_csv('eventsInstalls_set7.csv', encoding = 'utf_8', index=False)

In [317]:
eventsInstallsVentUnoRegressionSet.head(2)

,ref_hash,events_v1,date_v1,wifi_v1,installs_v1
0,41863526108385,88.0,76310.326,0.0,2.0
1,161514654074162,8.0,259200.000,0.0,0.0


In [318]:
installsV_1_2_3 = pd.merge(eventsInstallsVentUnoRegressionSet, eventsInstallsVentDosRegressionSet, \
                           on = 'ref_hash', how = 'outer')

In [319]:
installsV_1_2_3 = pd.merge(installsV_1_2_3, eventsInstallsVentTresRegressionSet, \
                           on = 'ref_hash', how = 'outer')

### Ahora para predecir el target

In [320]:
installsV_5_6_7 = pd.merge(eventsInstallsVentCincoRegressionSet, eventsInstallsVentSeisRegressionSet, \
                           on = 'ref_hash', how = 'outer')

In [321]:
installsV_5_6_7 = pd.merge(installsV_5_6_7, eventsInstallsVentSieteRegressionSet, \
                           on = 'ref_hash', how = 'outer')

In [322]:
installsV_1_2_3.head(2)

,ref_hash,events_v1,date_v1,wifi_v1,installs_v1,events_v2,date_v2,wifi_v2,installs_v2,events_v3,date_v3,wifi_v3,installs_v3
0,41863526108385,88.0,76310.326,0.0,2.0,72.0,259200.0,0.0,0.0,51.0,259200.000,0.0,0.0
1,161514654074162,8.0,259200.000,0.0,0.0,8.0,259200.0,0.0,0.0,51.0,238424.805,1.0,1.0


In [414]:
targetInstallV_4 = eventsInstallsVentCuatroRegressionSet[['ref_hash', 'date_v4']]

In [323]:
targetInstallV_6 = eventsInstallsVentSeisRegressionSet[['ref_hash', 'date_v6']]

In [478]:
targetInstallV_7 = eventsInstallsVentSieteRegressionSet[['ref_hash', 'date_v7']]

In [324]:
trainInstallsV_1_2_3_6 = pd.merge(installsV_1_2_3, targetInstallV_6, on = "ref_hash", how = "left")

In [325]:
trainInstallsV_1_2_3_6.head(2)

,ref_hash,events_v1,date_v1,wifi_v1,installs_v1,events_v2,date_v2,wifi_v2,installs_v2,events_v3,date_v3,wifi_v3,installs_v3,date_v6
0,41863526108385,88.0,76310.326,0.0,2.0,72.0,259200.0,0.0,0.0,51.0,259200.000,0.0,0.0,NaN
1,161514654074162,8.0,259200.000,0.0,0.0,8.0,259200.0,0.0,0.0,51.0,238424.805,1.0,1.0,259200.0


In [326]:
trainInstallsV_1_2_3_6[['date_v1', 'date_v2', 'date_v3', 'date_v6']] = \
trainInstallsV_1_2_3_6[['date_v1', 'date_v2', 'date_v3', 'date_v6']].fillna(value = 259200)

In [327]:
trainInstallsV_1_2_3_6[['installs_v1', 'installs_v2', 'installs_v3']] = \
trainInstallsV_1_2_3_6[['installs_v1', 'installs_v2', 'installs_v3']].fillna(value = 0)

In [333]:
trainInstallsV_1_2_3_6[['wifi_v1', 'wifi_v2', 'wifi_v3']] = \
trainInstallsV_1_2_3_6[['wifi_v1', 'wifi_v2', 'wifi_v3']].fillna(value = 0)

In [328]:
trainInstallsV_1_2_3_6[['events_v1', 'events_v2', 'events_v3']] = \
trainInstallsV_1_2_3_6[['events_v1', 'events_v2', 'events_v3']].fillna(value = 0)

In [477]:
trainInstallsV_1_2_3_6.head(10)

,ref_hash,events_v1,date_v1,wifi_v1,installs_v1,events_v2,date_v2,wifi_v2,installs_v2,events_v3,date_v3,wifi_v3,installs_v3,date_v6
0,41863526108385,88.0,76310.326,0.0,2.0,72.0,259200.000,0.0,0.0,51.0,259200.000,0.0,0.0,259200.0
1,161514654074162,8.0,259200.000,0.0,0.0,8.0,259200.000,0.0,0.0,51.0,238424.805,1.0,1.0,259200.0
2,186034136943920,13.0,60042.839,0.0,1.0,0.0,259200.000,0.0,0.0,28.0,259200.000,0.0,0.0,259200.0
3,360710529886978,3.0,259200.000,0.0,0.0,4.0,259200.000,0.0,0.0,5.0,259200.000,0.0,0.0,259200.0
4,365882020742330,57.0,96156.508,0.0,1.0,65.0,9756.508,0.0,1.0,41.0,259200.000,0.0,0.0,259200.0
5,475635010681369,17.0,259200.000,0.0,0.0,35.0,253871.662,1.0,1.0,26.0,167471.662,1.0,1.0,259200.0
6,501790157110512,2.0,259200.000,0.0,0.0,1.0,259200.000,0.0,0.0,1.0,259200.000,0.0,0.0,259200.0
7,519199987760489,2.0,92860.475,1.0,1.0,2.0,6460.475,1.0,1.0,0.0,259200.000,0.0,0.0,259200.0
8,558877640599287,1.0,65400.392,1.0,1.0,0.0,259200.000,0.0,0.0,0.0,259200.000,0.0,0.0,259200.0
9,643594200494946,7.0,259200.000,0.0,0.0,4.0,259200.000,0.0,0.0,2.0,259200.000,0.0,0.0,259200.0


In [415]:
trainInstallsV_1_4 = pd.merge(eventsInstallsVentUnoRegressionSet, targetInstallV_4, on = 'ref_hash', how = 'left')

In [416]:
trainInstallsV_1_4.head(3)

,ref_hash,events_v1,date_v1,wifi_v1,installs_v1,date_v4
0,41863526108385,88.0,76310.326,0.0,2.0,259200.000
1,161514654074162,8.0,259200.000,0.0,0.0,152024.805
2,186034136943920,13.0,60042.839,0.0,1.0,259200.000


In [417]:
trainInstallsV_1_4['date_v4'] = trainInstallsV_1_4['date_v4'].fillna(value = 259200)

In [418]:
trainInstallsV_1_4.count()

ref_hash       207964
events_v1      207964
date_v1        207964
wifi_v1        207964
installs_v1    207964
date_v4        207964
dtype: int64

### para el target

In [329]:
installsV_5_6_7[['date_v5', 'date_v6', 'date_v7']] = \
installsV_5_6_7[['date_v5', 'date_v6', 'date_v7']].fillna(value = 259200)

In [330]:
installsV_5_6_7[['installs_v5', 'installs_v6', 'installs_v7']] = \
installsV_5_6_7[['installs_v5', 'installs_v6', 'installs_v7']].fillna(value = 0)

In [331]:
installsV_5_6_7[['events_v5', 'events_v6', 'events_v7']] = \
installsV_5_6_7[['events_v5', 'events_v6', 'events_v7']].fillna(value = 0)

In [334]:
installsV_5_6_7[['wifi_v5', 'wifi_v6', 'wifi_v7']] = \
installsV_5_6_7[['wifi_v5', 'wifi_v6', 'wifi_v7']].fillna(value = 0)

In [335]:
installsV_5_6_7.head(2)

,ref_hash,events_v5,date_v5,wifi_v5,installs_v5,events_v6,date_v6,wifi_v6,installs_v6,events_v7,date_v7,wifi_v7,installs_v7
0,40621409780134,3.0,259200.0,0.0,0.0,3.0,259200.0,0.0,0.0,0.0,259200.0,0.0,0.0
1,69039685746313,4.0,259200.0,0.0,0.0,6.0,259200.0,0.0,0.0,16.0,259200.0,0.0,0.0


### Dejamos el id afuera

In [338]:
trainInstallsV_1_2_3_6sinId = trainInstallsV_1_2_3_6.drop(columns = ['ref_hash'])

In [339]:
installsV_5_6_7sinId = installsV_5_6_7.drop(columns = ['ref_hash'])

In [419]:
trainInstallsV_1_4sinId = trainInstallsV_1_4.drop(columns = ['ref_hash'])

### usamos el fit de n para n+3

In [480]:
arrayInstallV_7 = eventsInstallsVentSieteRegressionSet.drop(columns = ['ref_hash'])

In [481]:
arrayInstallV_7.head(2)

,events_v7,date_v7,wifi_v7,installs_v7
0,57.0,259200.0,0.0,0.0
1,16.0,259200.0,0.0,0.0


In [482]:
arrayInstallV_7 = arrayInstallV_7.values

### Forma NumPy

In [340]:
arraytrainInstallsV_1_2_3_6sinId = trainInstallsV_1_2_3_6sinId.values

In [420]:
arraytrainInstallsV_1_4sinId = trainInstallsV_1_4sinId.values

### Misma forma para el que necesitamos para predecir el target

In [341]:
X_arrayinstallsV_5_6_7sinId = installsV_5_6_7sinId.values

In [342]:
X_arraytrainInstallsV_1_2_3_6sinId = arraytrainInstallsV_1_2_3_6sinId[:, :-1]
y_arraytrainInstallsV_1_2_3_6sinId = arraytrainInstallsV_1_2_3_6sinId[:, -1]

In [421]:
X_arraytrainInstallsV_1_4sinId = arraytrainInstallsV_1_4sinId[:, :-1]
y_arraytrainInstallsV_1_4sinId = arraytrainInstallsV_1_4sinId[:, -1]

In [343]:
X_train, X_test, y_train, y_test = train_test_split(X_arraytrainInstallsV_1_2_3_6sinId,\
                                                   y_arraytrainInstallsV_1_2_3_6sinId, test_size=0.2, random_state=42)

In [483]:
X_train, X_test, y_train, y_test = train_test_split(X_arraytrainInstallsV_1_4sinId,\
                                                   y_arraytrainInstallsV_1_4sinId, test_size=0.2, random_state=42)

### Con regresion

In [344]:
regTrainingSetV_1_2_3_6 = LinearRegression().fit(X_train, y_train)

In [484]:
regTrainingSetV_1_4 = LinearRegression().fit(X_train, y_train)

In [345]:
y_estim = regTrainingSetV_1_2_3_6.predict(X_test)

In [485]:
y_estim = regTrainingSetV_1_4.predict(X_test)

In [347]:
rmseRegLin = math.sqrt(mean_squared_error(y_test, y_estim))

In [486]:
rmseRegLinV_1_4 = math.sqrt(mean_squared_error(y_test, y_estim))

In [348]:
rmseRegLin

33093.558759031606

In [487]:
rmseRegLinV_1_4

36179.697956423355

In [409]:
regTrainingSetV_1_2_3_6.score(X_test, y_test)

0.016397678190139264

In [427]:
regTrainingSetV_1_4.score(X_test, y_test)

0.013753025752257675

In [410]:
regTrainingSetV_1_2_3_6.coef_

array([-6.00815266e+00, -5.13836783e-02, -9.74919542e+02, -1.25038542e+03,
        4.00345332e+00, -1.78686335e-02,  5.10499450e+01,  5.45622927e+02,
       -2.05105175e+01,  1.50191136e-03, -5.76692622e+02,  7.41948369e+03])

In [428]:
regTrainingSetV_1_4.coef_

array([-1.41476286e+01, -1.60183969e-02, -1.05453993e+03,  7.79292559e+03])

### GridSearch con XGBoost para buscar los mejores parámetros. 

In [469]:
xgb1 = xgboost.XGBRegressor()
parameters = { 
                      'objective':['reg:linear'],
                      'learning_rate': [0.045,0.05,0.06], 
                      'max_depth': [5],
                      'min_child_weight': [2,3,4],
                      'silent': [1],
                      'subsample': [0.5,0.55,0.6],
                      'colsample_bytree': [0.85],
                      'n_estimators': [650]}

xgb_grid = GridSearchCV(xgb1, parameters, scoring='neg_mean_squared_error', #this line can be commented to use XGB's default metric
                            cv = 5,
                            n_jobs = 5,
                            verbose=True)


xgb_grid.fit(X_train,y_train)
    

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed: 17.0min
[Parallel(n_jobs=5)]: Done 135 out of 135 | elapsed: 52.9min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=5,
       param_grid={'objective': ['reg:linear'], 'learning_rate': [0.045, 0.05, 0.06], 'max_depth': [5], 'min_child_weight': [2, 3, 4], 'silent': [1], 'subsample': [0.5, 0.55, 0.6], 'colsample_bytree': [0.85], 'n_estimators': [650]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=True)

In [470]:
y_estim = xgb_grid.predict(X_test)

In [471]:
rmseXGBgrid = math.sqrt(mean_squared_error(y_test, y_estim))

In [472]:
rmseXGBgrid

36139.51417377929

In [473]:
print(explained_variance_score(y_estim ,y_test))

-33.09458698278196


### Con ese fit aplicado a V_1_4, calcularemos una prediccion para el target.

### Aca probamos otros fits con otros input y otros hiperparametros.

In [350]:
xgbV_1_2_3_6 = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')

In [488]:
xgbV_1_4 = xgboost.XGBRegressor(n_estimators=100, learning_rate=1, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7, objective='reg:squarederror')

In [351]:
xgbV_1_2_3_6.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.08, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=0.75, verbosity=1)

In [489]:
xgbV_1_4.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=0.75, verbosity=1)

In [352]:
y_estim = xgbV_1_2_3_6.predict(X_test)

In [490]:
y_estim = xgbV_1_4.predict(X_test)

In [411]:
y_train_estim = xgbV_1_2_3_6.predict(X_train)

In [353]:
rmseXGB = math.sqrt(mean_squared_error(y_test, y_estim))

In [491]:
rmseXGBV_1_4 = math.sqrt(mean_squared_error(y_test, y_estim))

In [354]:
rmseXGB

32980.77870462715

In [492]:
rmseXGBV_1_4

37803.02571606861

In [458]:
print(explained_variance_score(y_estim ,y_test))

-7.754393252529219


In [412]:
print(explained_variance_score(y_train_estim , y_train))

-28.68581495695423


In [408]:
print(explained_variance_score(y_estim ,y_test))

-48.633896279367676


### Con Ridge alpha = 1

In [355]:
RidgeV_1_2_3_6 = Ridge(alpha = 10)

In [356]:
RidgeV_1_2_3_6.fit(X_train, y_train)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [357]:
y_estim = RidgeV_1_2_3_6.predict(X_test)

In [358]:
rmseRidge1 = math.sqrt(mean_squared_error(y_test, y_estim))

In [359]:
rmseRidge1

33093.55312595675

### Usamos el fitter de XGBoost para predecir el target

In [360]:
y_targetXGBoost = xgbV_1_2_3_6.predict(X_arrayinstallsV_5_6_7sinId)

In [363]:
df_device_installsV_5_6_7 = installsV_5_6_7[['ref_hash']]

In [364]:
df_device_installsV_5_6_7 = df_device_installsV_5_6_7.reset_index(drop = True)

### otro fit

In [493]:
y_targetXGBoost = xgbV_1_4.predict(arrayInstallV_7)

In [494]:
dfPredictXGBoost = pd.DataFrame(y_targetXGBoost, columns = ['prediccion_install'])

In [495]:
df_device_installsV_7 = eventsInstallsVentSieteRegressionSet[['ref_hash']]

In [496]:
df_device_installsV_7 = df_device_installsV_7.reset_index(drop = True)

In [497]:
dfPredictXGBoost = pd.concat([df_device_installsV_7, dfPredictXGBoost], axis = 1)

In [498]:
len(dfPredictXGBoost)

241298

In [499]:
dfPredictXGBoost['ref_hash'] = dfPredictXGBoost.ref_hash.astype(str) + '_sc'

In [500]:
dfPredictXGBoost.columns = ['ref_hash', 'prediccion_install']

In [501]:
dfPredictXGBoost.head(2)

,ref_hash,prediccion_install
0,41863526108385_sc,256103.15625
1,69039685746313_sc,245650.12500


## Ahora a leer el archivo llenado con los Sc, para ponerle los valores St

In [502]:
target = pd.read_csv('./targetConScConSt.csv')

In [503]:
target.head()

,ref_hash,obj
0,1000169251625791246_sc,259200
1,1000169251625791246_st,73579
2,1000395625957344683_sc,161908
3,1000395625957344683_st,46671
4,1003027494996471685_sc,182100


In [504]:
target.loc[target.index % 2 == 0, 'obj'] = 0

In [505]:
target = pd.merge(target, dfPredictXGBoost, on = 'ref_hash', how = 'left')

In [506]:
target.count()

ref_hash              8074
obj                   8074
prediccion_install    2394
dtype: int64

In [507]:
target.head(10)

,ref_hash,obj,prediccion_install
0,1000169251625791246_sc,0,244028.765625
1,1000169251625791246_st,73579,NaN
2,1000395625957344683_sc,0,NaN
3,1000395625957344683_st,46671,NaN
4,1003027494996471685_sc,0,244132.734375
5,1003027494996471685_st,118291,NaN
6,1006670001679961544_sc,0,NaN
7,1006670001679961544_st,150630,NaN
8,1007573308966476713_sc,0,NaN
9,1007573308966476713_st,95972,NaN


In [508]:
target.loc[target.index % 2 != 0, 'prediccion_install'] = 0

In [509]:
target.head(4)

,ref_hash,obj,prediccion_install
0,1000169251625791246_sc,0,244028.765625
1,1000169251625791246_st,73579,0.000000
2,1000395625957344683_sc,0,NaN
3,1000395625957344683_st,46671,0.000000


In [510]:
target = target.fillna(value = 259200)

In [511]:
target.head(4)

,ref_hash,obj,prediccion_install
0,1000169251625791246_sc,0,244028.765625
1,1000169251625791246_st,73579,0.000000
2,1000395625957344683_sc,0,259200.000000
3,1000395625957344683_st,46671,0.000000


In [512]:
target['prediccion_install'] = target.prediccion_install.round().astype(int)

In [513]:
target['obj'] = target['obj'] + target['prediccion_install']

In [514]:
target.head(2)

,ref_hash,obj,prediccion_install
0,1000169251625791246_sc,244029,244029
1,1000169251625791246_st,73579,0


In [515]:
target = target.drop(columns = ['prediccion_install'])

In [516]:
target.head()

,ref_hash,obj
0,1000169251625791246_sc,244029
1,1000169251625791246_st,73579
2,1000395625957344683_sc,259200
3,1000395625957344683_st,46671
4,1003027494996471685_sc,244133


In [517]:
target.to_csv('targetInstallXGBoostv_1_4.csv', encoding='utf-8', index=False)

In [325]:
targetUpdated.to_csv('targetConSc.csv', encoding='utf-8', index=False)